In [1]:
import enum

from sqlalchemy import (
    Column,
    create_engine,
    Enum,
    ForeignKey,
    Integer,
    String,
    Table,
)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

import vocalpy as voc

In [2]:
Base = declarative_base()

/var/folders/th/rsbsynzn56zbz5vrtgv63yh00000gn/T/ipykernel_14354/4196137762.py:1: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
class AudioFile(Base):
    __tablename__ = 'audio_files'
    
    id = Column(Integer, primary_key=True)
    dataset_file_id = Column(Integer, ForeignKey('dataset_files.id'))
    
    dataset_file = relationship("DatasetFile", back_populates="audio_file")
    spectrogram_files = relationship("SpectrogramFile", back_populates="source_audio_file")

    def __repr__(self):
        return f"AudioFile(id={self.id!r}, dataset_file_id={self.dataset_file_id!r})"

In [4]:
annotates_table = Table(
    "annotates",
    Base.metadata,
    Column("annotation_file_id", ForeignKey("annotation_files.id")),
    Column("dataset_file_id", ForeignKey("dataset_files.id")),
)

In [5]:
class AnnotationFile(Base):
    __tablename__ = 'annotation_files'
    
    id = Column(Integer, primary_key=True)
    dataset_file_id = Column(Integer, ForeignKey('dataset_files.id'))

    annotates = relationship("DatasetFile", secondary=annotates_table)

    dataset_file = relationship("DatasetFile", back_populates="annotation_file")
    
    def __repr__(self):
        return f"AnnotationFile(id={self.id!r}, dataset_file_id={self.dataset_file_id!r}"

In [6]:
class SpectrogramFile(Base):
    __tablename__ = 'spectrogram_files'
    
    id = Column(Integer, primary_key=True)
    source_audio_file_id = Column("audio_file_id", Integer, ForeignKey("audio_files.id"))
    spectrogram_parameters_id = Column("spectrogram_parameters_id", Integer, ForeignKey("spectrogram_parameters.id"))
    dataset_file_id = Column(Integer, ForeignKey('dataset_files.id'))

    source_audio_file = relationship("AudioFile", back_populates="spectrogram_files")
    spectrogram_parameters = relationship("SpectrogramParameters", back_populates="spectrogram_files")
    dataset_file = relationship("DatasetFile", back_populates="spectrogram_file")
    
    def __repr__(self):
        return (f"SpectrogramFile(id={self.id!r}, source_audio_file={self.source_audio_file!r}, "
                f"spectrogram_parameters={self.spectrogram_parameters!r})")

In [7]:
class SpectrogramParameters(Base):
    __tablename__ = 'spectrogram_parameters'
    
    id = Column(Integer, primary_key=True)
    fft_size = Column(Integer)
    step_size = Column(Integer)
    
    spectrogram_files = relationship("SpectrogramFile", back_populates="spectrogram_parameters")
    
    def __repr__(self):
        return f"SpectrogramParameters(id={self.id!r}, fft_size={self.fft_size!r}, step_size={self.step_size!r}"

In [8]:
class DatasetFileTypeEnum(enum.Enum):
    AudioFile = 1
    SpectrogramFile = 2
    AnnotationFile = 3
    FeatureFile = 4

In [9]:
class DatasetFile(Base):
    __tablename__ = 'dataset_files'
    
    id = Column(Integer, primary_key=True)
    path = Column(String)
    file_type = Column(Enum(DatasetFileTypeEnum))

    audio_file = relationship("AudioFile", 
                              back_populates="dataset_file", 
                              uselist=False)
    spectrogram_file = relationship("SpectrogramFile", 
                                    back_populates="dataset_file", 
                                    uselist=False)

    annotation_file = relationship("AnnotationFile", 
                                    back_populates="dataset_file", 
                                    uselist=False)

In [10]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [11]:
def get_engine():
     return create_engine('sqlite:///:memory:', echo=True)

def get_session(engine):
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

def an_audio_path():
    return './tests/data/source/bird1.cbin'

In [2]:
from sqlalchemy.orm import registry
from sqlalchemy import Table, Column, Integer, Enum, String

mapper_registry = registry()



In [ ]:
engine = get_engine()
session = get_session(engine)

# next line makes tables?
Base.metadata.create_all(engine)

In [12]:
audio_path = an_audio_path()

dataset_audio_file = DatasetFile(path=audio_path, file_type='AudioFile')
session.add(dataset_audio_file)

audio_file = AudioFile(dataset_file=dataset_audio_file)
session.add(audio_file)

2023-04-18 17:20:59,890 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-18 17:20:59,891 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("audio_files")
2023-04-18 17:20:59,892 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 17:20:59,894 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("audio_files")
2023-04-18 17:20:59,895 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 17:20:59,896 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("annotates")
2023-04-18 17:20:59,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 17:20:59,901 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("annotates")
2023-04-18 17:20:59,902 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 17:20:59,904 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("annotation_files")
2023-04-18 17:20:59,905 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-18 17:20:59,907 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("annotation_files")
2023-04-18 17:20:59,908 INFO sqlalch

In [13]:
spect_params = SpectrogramParameters(fft_size=512, step_size=64)

In [14]:
spect_path = './tests/data/source/bird1.npz'
dataset_spect_file = DatasetFile(path=spect_path, file_type='SpectrogramFile')
session.add(dataset_spect_file)

spect_file = SpectrogramFile(dataset_file=dataset_spect_file, spectrogram_parameters=spect_params, source_audio_file=audio_file)
session.add(spect_file)

In [15]:
annot_path = './tests/data/source/bird1.not.mat'
dataset_annot_file = DatasetFile(path=annot_path, file_type='AnnotationFile')
session.add(dataset_annot_file)

annot_file = AnnotationFile(dataset_file=dataset_annot_file, annotates=[dataset_audio_file])
session.add(annot_file)

In [16]:
queried_audio_file = session.query(AudioFile).first()

2023-04-18 17:21:00,063 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-18 17:21:00,067 INFO sqlalchemy.engine.Engine INSERT INTO spectrogram_parameters (fft_size, step_size) VALUES (?, ?)
2023-04-18 17:21:00,068 INFO sqlalchemy.engine.Engine [generated in 0.00126s] (512, 64)
2023-04-18 17:21:00,071 INFO sqlalchemy.engine.Engine INSERT INTO dataset_files (path, file_type) VALUES (?, ?)
2023-04-18 17:21:00,072 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ('./tests/data/source/bird1.cbin', 'AudioFile')
2023-04-18 17:21:00,074 INFO sqlalchemy.engine.Engine INSERT INTO dataset_files (path, file_type) VALUES (?, ?)
2023-04-18 17:21:00,075 INFO sqlalchemy.engine.Engine [cached since 0.003377s ago] ('./tests/data/source/bird1.npz', 'SpectrogramFile')
2023-04-18 17:21:00,076 INFO sqlalchemy.engine.Engine INSERT INTO dataset_files (path, file_type) VALUES (?, ?)
2023-04-18 17:21:00,076 INFO sqlalchemy.engine.Engine [cached since 0.005357s ago] ('./tests/data/source/bird1.not.mat

In [17]:
queried_audio_file

AudioFile(id=1, dataset_file_id=1)

In [18]:
queried_spect_file = session.query(SpectrogramFile).first()

2023-04-18 17:21:01,860 INFO sqlalchemy.engine.Engine SELECT spectrogram_files.audio_file_id AS spectrogram_files_audio_file_id, spectrogram_files.id AS spectrogram_files_id, spectrogram_files.spectrogram_parameters_id AS spectrogram_files_spectrogram_parameters_id, spectrogram_files.dataset_file_id AS spectrogram_files_dataset_file_id 
FROM spectrogram_files
 LIMIT ? OFFSET ?
2023-04-18 17:21:01,862 INFO sqlalchemy.engine.Engine [generated in 0.00225s] (1, 0)


In [19]:
queried_spect_file

SpectrogramFile(id=1, source_audio_file=AudioFile(id=1, dataset_file_id=1), spectrogram_parameters=SpectrogramParameters(id=1, fft_size=512, step_size=64)

In [20]:
session.commit()

2023-04-18 17:52:24,722 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:



def test_dataset_file():
    engine = get_engine()
    session = get_session(engine)

    # next line makes tables?
    Base.metadata.create_all(engine)
    
    path = an_audio_path()
    
    test_file = DatasetFile(path=path, file_type='AudioFile')
    session.add(test_file)

    queried_file = session.query(DatasetFile).filter_by(path=path).first()
    assert queried_file.path == path
    assert queried_file.file_type == 'AudioFile'
    assert queried_file.id == 1

    Base.metadata.drop_all(engine)

In [ ]:
test_dataset_file()

In [ ]:
def test_audio_file():
    engine = get_engine()
    session = get_session(engine)

    # next line makes tables?
    Base.metadata.create_all(engine)
    
    path = an_audio_path()
    
    test_dataset_file = DatasetFile(path=path, file_type='AudioFile')
    session.add(test_dataset_file)

    test_audio_file = AudioFile(dataset_file=test_dataset_file)
    session.add(test_audio_file)
    
    queried_file = session.query(
        AudioFile
    ).filter_by(
        id=1
    ).first()
    assert queried_file.path == path
    assert queried_file.file_type == 'AudioFile'
    assert queried_file.id == 1

    Base.metadata.drop_all(engine)

In [ ]:
test_audio_file()